# Definición del problema

Se desea entrenar un modelo que sea capaz de 'completar' una palabra a medio escribir, o proponer una corrección para una palabra ya escrita en caso de que la misma se encuentre mal escrita.

Se utilizará un algoritmo de 'hallar la palabra incorrecta' para determinar si una palabra está escrita incorrectamente, de acuerdo a un lexicón construido con palabras extraídas de la página web de la RAE (disponible en https://github.com/JorgeDuenasLerin/diccionario-espanol-txt, actualizado en Mayo 2024)

Además, se construirá una matriz de probabilidad con las palabras extraídas para que las recomendaciones de completado y corrección se realicen en función de la frecuencia de utilización de las palabras. El sistema será capaz de realizar estas funciones en Español.

Se utilizarán textos para entrenarlo.

# Estructura del modelo

# Datos de entrada

In [34]:
#Extrayendo el texto
txt_file = "text_dump.txt"

lines: list[str] = []
# complete_text: str = ""
with open(txt_file, "r") as file:
  for line in file:
    if line != "\n":
      lines.append(line)
      # complete_text += line

# print(len(lines))
# print(complete_text)

753


In [ ]:
#Formando la data de entrada y salida esperada
complete_text: str = ""
regex_specials = """-—?¿"'!¡:;.,()…“”/"""


for paragraph in lines[6 : 747]:
  clean_paragraph = ""
  for char in paragraph:
    if char not in regex_specials:
      clean_paragraph += char

  complete_text += clean_paragraph

complete_text = complete_text.lower()


#Construyendo la data de input, output y conjunto de tokens
encoder_tokens: dict[str, int] = dict()
decoder_tokens: dict[str, int] = dict()
input_chars: list[str] = []
output_chars: list[str] = []


# input_chars.append(complete_text[0])

#Llenar los inputs y outputs de manera escalonada
for i in range(0, len(complete_text) - 1):
  input_char = complete_text[i]

  input_chars.append(input_char)
  output_chars.append(complete_text[i + 1])

  if input_char not in encoder_tokens:
    index = len(encoder_tokens)
    encoder_tokens[input_char] = index
    decoder_tokens[index] = input_char


# for i, pair in enumerate(zip(input_chars, output_chars)):
#   print(pair)
#   index += 1
#   if index >= 100:
#     break

# for key, value in zip(encoder_tokens.keys(), encoder_tokens.values()):
#   print(f"Key: {key} | Value: {value}")

In [13]:
import numpy as np
# Creando la matriz de caracteres

input_size = len(input_chars)
token_size = len(encoder_tokens)

input_data = np.zeros((input_size, 1,token_size), dtype= "float32")
# decoder_input_data = np.zeros((input_size, token_size), dtype= "float32")
output_data = np.zeros((input_size, 1,token_size), dtype= "float32")


for i, (input, output) in enumerate(zip(input_chars, output_chars)):
  input_data[i, 0,encoder_tokens[input]] = 1.0
  output_data[i, 0,encoder_tokens[output]] = 1.0

# Construción del modelo

In [36]:
import tensorflow as tf
from tensorflow import keras
from keras import initializers

neurons = 800

In [37]:
weight_initializer = initializers.RandomNormal(mean= 0.01, stddev= 0.08)

encoder_inputs = keras.Input(shape= (None, token_size))
encoder = keras.layers.LSTM(neurons, return_state= True, kernel_initializer= weight_initializer)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape= (None, token_size))

decoder_lstm = keras.layers.LSTM(neurons, return_sequences= True, return_state= True, kernel_initializer= weight_initializer)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense = keras.layers.Dense(token_size, activation= "softmax", kernel_initializer= weight_initializer )
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer= "rmsprop", loss= "categorical_crossentropy", metrics= ["accuracy"])

# Entrenamiento del modelo

In [38]:
model.fit([input_data, input_data], output_data, batch_size= 64, epochs= 200, validation_split= 0.3)

model.save("s2s-Principito.keras")

Epoch 1/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.2559 - loss: 2.6313 - val_accuracy: 0.2786 - val_loss: 2.3074
Epoch 2/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.2805 - loss: 2.2940 - val_accuracy: 0.2759 - val_loss: 2.2982
Epoch 3/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.2760 - loss: 2.2906 - val_accuracy: 0.2738 - val_loss: 2.2921
Epoch 4/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.2750 - loss: 2.2888 - val_accuracy: 0.2762 - val_loss: 2.2907
Epoch 5/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.2801 - loss: 2.2770 - val_accuracy: 0.2792 - val_loss: 2.2903
Epoch 6/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.2734 - loss: 2.2901 - val_accuracy: 0.2734 - val_loss: 2.2925
Epoch 7/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2776 - loss: 2.2865 - val_accuracy: 0.2664 - val_loss: 2.2943
Epoch 8/200
813/813 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.2778 - loss: 2.2787 - val_ac

# Evaluación

In [39]:
model = keras.models.load_model("s2s-Principito.keras")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_inputs = [keras.Input(shape= (neurons,)), keras.Input(shape= (neurons,))]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state= decoder_state_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

def decode_word(input_char: str) -> str:
  data = np.zeros((1, 1, token_size))
  data[0, 0, encoder_tokens[input_char]] = 1.0

  state_value = encoder_model.predict(data)

  target_sequence = np.zeros((1, 1, token_size))
  target_sequence[0,0, encoder_tokens[input_char]] = 1.0

  decoded_word = input_char
  sampled_char = ""
  failsafe = 0
  while sampled_char != " " and failsafe < 25:
    failsafe += 1
    output_tokens, h, c = decoder_model.predict([target_sequence] + state_value)

    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = decoder_tokens[sampled_token_index]
    decoded_word += sampled_char

    target_sequence = np.zeros((1, 1, token_size))
    target_sequence[0, 0, sampled_token_index] = 1.0

    state_value = [h, c]

  return decoded_word


# sequence = input_data[1 : 2]
# print(decoder_tokens[np.argmax(input_data[1, -1, :])])

# # Define the shape of the decoder input explicitly
# decoder_input_shape = (sequence.shape[0], sequence.shape[1], token_size)
# # Assuming token_size is defined in your code

# # Create the decoder input with the defined shape
# decoder_input = output_data[1 : 2]
# i = 0
# decoded_char = ""
# while decoded_char != " " and i < 25:
#   i += 1
#   # Now, predict using the model
#   output_tokens = model.predict([sequence, decoder_input])

#   token_index = np.argmax(output_tokens[0, -1, :])

#   decoded_char = decoder_tokens[token_index]

#   print(decoded_char)

#   sequence[0, 0, token_index] = 1.0

In [44]:
predicted_word = decode_word("j")

print(predicted_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
jo1hhh 
